# Recipe 01: Return file content

### Use case: 
Receive HTTP Resuest, read file content, then return content as the HTTP Response

### Reference:

* Exposing data in a RESTful API with NiFi, https://www.clearpeaks.com/exposing-data-in-a-restful-api-with-nifi/

### NiFi Examples: (https://bolsrcdnifi01.micron.com:8444/nifi/?processGroupId=fcbb344d-2b99-1f42-0000-0000556af466&componentIds=)

* StandardHttpContextMap4Test
  * Setting
    * Type: StandardHttpContextMap 1.11.4
  * Properties
    * Max Oustanding Requests: 5000
    * Request Expiration: 1 min

* HandleHttpRequest
  * Setting
    * Type: HandleHttpRequest 1.11.4
  * Properties
    * Listening Port: 58001
    * HTTP Context Map: StandardHttpContextMap4Test
    * Allowed Paths: /ecstoken
    * Client Authentication: No Authentication
  * Relationships
    * Success: FetchFile

* FetchFile
  * Setting
    * Type: FetchFile 1.11.4
  * Properties
    * File to Fetch: /tmp/date.log
    * Completion Strategy: None
  * Relationships
    * Success: FetchFile
    * failure/not.found/permission.denied: Funnel

* HandleHttpResponse
  * Setting
    * Type: HandleHttpResponse 1.11.4
  * Properties
    * HTTP Status Code: 200
    * HTTP Context Map: StandardHttpContextMap4Leo
  * Relationships
    * Success: Funnel

<img src="01_01.png" style="display:block;margin-left:0;height:300px;"/>


### Python Examples

In [ ]:
import requests

apihost="bolsrcdnifi01.micron.com:58001"
def getAccessToken(encodedCred):
    session = requests.Session()
    url = "http://{_apihost}/ecstoken".format(_apihost=apihost)
    headers = {'Authorization': 'Basic ' + encodedCred} 
    data = {
        "grant_type":"client_credentials"
    }
    response = requests.get(url, headers=headers, data=data, verify=False)
    #accessToken = response.json()
    return response


output:
```
RESP Status Code:200
RESP Text:Wed Nov 18 05:13:52 GMT 2020
```

In [ ]:
resp = getAccessToken("abc")
print("RESP Status Code:{_code}".format(_code=resp.status_code))
print("RESP Text:{_text}".format(_text=resp.text))